In [2]:
import numpy as np
import torch

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1

yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2


## Training data

We can represent the training data using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [4]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [5]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [6]:
# Weights and biases
# since we don't know what the real values are we use random to train

w = torch.randn(2, 3, requires_grad=True)
# 2: apples and oranges
# 3: each have 3 coeff
b = torch.randn(2, requires_grad=True)
# bias is const
print(w)
print(b)

tensor([[ 0.9887, -0.6828, -0.3289],
        [-0.0954, -0.1854, -0.6360]], requires_grad=True)
tensor([1.0996, 0.1550], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [7]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [9]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 13.3850, -46.5741],
        [  9.9359, -65.5391],
        [-23.4534, -69.8682],
        [ 60.4180, -41.0758],
        [-19.2516, -68.7394]], grad_fn=<AddBackward0>)


In [10]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [11]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [12]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(16740.2676, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [13]:
# Compute gradients
loss.backward()

In [14]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.9887, -0.6828, -0.3289],
        [-0.0954, -0.1854, -0.6360]], requires_grad=True)
tensor([[ -5297.5786,  -7656.3545,  -4355.7974],
        [-12446.4512, -14206.0977,  -8693.6211]])


## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [15]:
w
w.grad

tensor([[ -5297.5786,  -7656.3545,  -4355.7974],
        [-12446.4512, -14206.0977,  -8693.6211]])

In [16]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [17]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)
# tensor(16740.2676, grad_fn=<DivBackward0>)

tensor(16740.2676, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [18]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)


tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [19]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 24.2557, -24.2304],
        [ 24.2827, -36.1460],
        [ -6.0579, -34.9598],
        [ 70.7261, -19.0537],
        [ -5.1964, -40.4264]], grad_fn=<AddBackward0>)


In [20]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(11840.2412, grad_fn=<DivBackward0>)


In [21]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -4170.8281,  -6433.4775,  -3603.8311],
        [-10137.9023, -11720.8301,  -7160.9517]])
tensor([ -54.5980, -122.9633])


In [22]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [23]:
print(w)
print(b)

tensor([[ 1.0834, -0.5419, -0.2493],
        [ 0.1305,  0.0739, -0.4774]], requires_grad=True)
tensor([1.1008, 0.1578], requires_grad=True)


In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(8531.3965, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [25]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [26]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(530.4518, grad_fn=<DivBackward0>)


In [27]:
# Predictions
preds

tensor([[ 66.6240,  74.9219],
        [ 84.7060,  96.7590],
        [ 97.8061, 134.4382],
        [ 74.5978,  62.9608],
        [ 75.2546,  97.2248]], grad_fn=<AddBackward0>)

In [29]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [28]:
# Train for another 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [30]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(184.3512, grad_fn=<DivBackward0>)


In [31]:
# Predictions
preds

tensor([[ 62.4512,  73.0744],
        [ 81.8864,  95.9940],
        [110.9274, 139.1482],
        [ 50.3475,  52.2725],
        [ 84.5003, 102.1311]], grad_fn=<AddBackward0>)

In [32]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [33]:
import torch.nn as nn

In [34]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [35]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

We are using 15 training examples to illustrate how to work with large datasets in small batches. 

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [36]:
from torch.utils.data import TensorDataset

In [37]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [39]:
train_ds[:]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [ 74.,  66.,  43.],
         [ 91.,  87.,  65.],
         [ 88., 134.,  59.],
         [101.,  44.,  37.],
         [ 68.,  96.,  71.],
         [ 73.,  66.,  44.],
         [ 92.,  87.,  64.],
         [ 87., 135.,  57.],
         [103.,  43.,  36.],
         [ 68.,  97.,  70.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 57.,  69.],
         [ 80., 102.],
         [118., 132.],
         [ 21.,  38.],
         [104., 118.],
         [ 57.,  69.],
         [ 82., 100.],
         [118., 134.],
         [ 20.,  38.],
         [102., 120.]]))

In [40]:
from torch.utils.data import DataLoader

In [41]:
# Define data loader
batch_size = 5
# each batch has 5 datas
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
# turn data into what we want

In [42]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break


tensor([[ 73.,  67.,  43.],
        [ 68.,  97.,  70.],
        [ 68.,  96.,  71.],
        [101.,  44.,  37.],
        [ 92.,  87.,  64.]])
tensor([[ 56.,  70.],
        [102., 120.],
        [104., 118.],
        [ 21.,  38.],
        [ 82., 100.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [45]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)
# they are random

Parameter containing:
tensor([[ 0.3738,  0.2517,  0.2002],
        [-0.5587, -0.5021,  0.2070]], requires_grad=True)
Parameter containing:
tensor([-0.0935,  0.0559], requires_grad=True)


In [46]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3738,  0.2517,  0.2002],
         [-0.5587, -0.5021,  0.2070]], requires_grad=True),
 Parameter containing:
 tensor([-0.0935,  0.0559], requires_grad=True)]

In [47]:
# Generate predictions
preds = model(inputs)
preds

tensor([[  52.6659,  -65.4698],
        [  68.8839,  -81.7240],
        [  77.7664, -103.8300],
        [  56.2636,  -70.8614],
        [  63.8753,  -72.2087],
        [  52.7880,  -65.5263],
        [  68.8324,  -81.0149],
        [  78.3404, -104.1816],
        [  56.1416,  -70.8049],
        [  63.7017,  -71.4430],
        [  52.6143,  -64.7606],
        [  69.0060,  -81.7805],
        [  77.8179, -104.5391],
        [  56.4372,  -71.6271],
        [  63.7532,  -72.1522]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [49]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [50]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [51]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(16054.3955, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [52]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [53]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [54]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 332.3311
Epoch [20/100], Loss: 271.2419
Epoch [30/100], Loss: 98.7683
Epoch [40/100], Loss: 91.8338
Epoch [50/100], Loss: 142.6706
Epoch [60/100], Loss: 38.2424
Epoch [70/100], Loss: 31.6665
Epoch [80/100], Loss: 5.1396
Epoch [90/100], Loss: 21.0058
Epoch [100/100], Loss: 14.7391


In [55]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.4012,  71.2793],
        [ 80.9873, 100.7215],
        [118.8219, 131.1865],
        [ 28.6751,  42.6090],
        [ 95.3958, 115.9820],
        [ 57.3145,  70.3304],
        [ 80.5465, 100.7884],
        [119.0078, 131.8084],
        [ 29.7618,  43.5579],
        [ 96.0417, 116.9977],
        [ 57.9604,  71.3461],
        [ 79.9006,  99.7727],
        [119.2627, 131.1196],
        [ 28.0293,  41.5933],
        [ 96.4825, 116.9309]], grad_fn=<AddmmBackward0>)

In [56]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [57]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.9302, 68.6964]], grad_fn=<AddmmBackward0>)

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)

Keep this picture in mind as you work through the next few tutorials. 
